In [46]:
import pandas as pd
import io
from gmap import GMap
import dotenv
import os
import datetime
import pickle
import subprocess
import pyperclip

## データの生成

生成済みのデータが存在する場合はスキップする


In [10]:
csv = """
京都駅, 0, 0
音羽山清水寺, 30, 3
伏見稲荷大社, 45, 3
金閣寺, 30, 3
二条城, 30, 3
八坂神社, 15, 1
銀閣寺, 30, 2
錦市場, 30, 3
京都御所, 15, 1
下鴨神社, 20, 2
上賀茂神社, 20, 2
東寺, 30, 2
三十三間堂, 20, 2
八坂庚申堂, 10, 2
安井金比羅宮, 20, 2
南禅寺, 30, 3
龍安寺, 30, 3
仁和寺, 20, 1
北野天満宮, 20, 2
京都水族館, 90, 3
京都タワー, 30, 2
京都国立博物館, 90, 2
京都鉄道博物館, 90, 2
西本願寺, 30, 2
六波羅蜜寺, 30, 1
京都大学中央食堂, 30, 100
""".strip()
data = pd.read_csv(io.StringIO(csv), header=None, names=[
                   'spot', 'stay_time', 'priority'])

In [11]:
START_TIME = 9*60*60

In [12]:
data["arrive_before"] = -1
data["arrive_after"] = -1
data["stay_time"] = data["stay_time"] * 60

In [13]:
dotenv.load_dotenv()
GOOGLE_MAPS_API_KEY = os.getenv("GOOGLE_MAPS_API_KEY")
if not GOOGLE_MAPS_API_KEY:
    raise Exception("GOOGLE_MAPS_API_KEY not found")
gmap = GMap(GOOGLE_MAPS_API_KEY)

In [14]:
spots = data["spot"].tolist()
n = len(spots)
d_matrix = [[0] * n for _ in range(n)]

for i in range(0, n + 1, 10):
    for j in range(0, n + 1, 10):
        i_end = min(i + 10, n)
        j_end = min(j + 10, n)
        tmp = gmap.distance_matrix(spots[i:i_end], spots[j:j_end])
        for k in range(i, i_end):
            for l in range(j, j_end):
                d_matrix[k][l] = tmp[k - i][l - j]

In [15]:
for spot in spots:
    location = gmap.get_location(spot)
    business_time = gmap.get_business_time(spot, datetime.datetime(2024, 1, 14))
    data.loc[data["spot"] == spot, "latitude"] = location["lat"]
    data.loc[data["spot"] == spot, "longitude"] = location["lng"]
    b_start, b_end = business_time
    if b_start is not None:
        hour = int(b_start[:2])
        minute = int(b_start[2:])
        start_sec = hour * 60 * 60 + minute * 60
        data.loc[data["spot"] == spot, "arrive_after"] = max(-1, start_sec - START_TIME)
    if b_end is not None:
        hour = int(b_end[:2])
        minute = int(b_end[2:])
        end_sec = hour * 60 * 60 + minute * 60
        data.loc[data["spot"] == spot, "arrive_before"] = end_sec - START_TIME

In [16]:
data.loc[data["spot"] == "京都駅", "arrive_before"] = 18 * 60 * 60 - START_TIME
data.loc[data["spot"] == "京都大学中央食堂",
         "arrive_before"] = 13 * 60 * 60 - START_TIME
data.loc[data["spot"] == "京都大学中央食堂",
         "arrive_after"] = 12 * 60 * 60 - START_TIME

In [17]:
data.to_pickle("data.pkl")
pickle.dump(d_matrix, open("d_matrix.pkl", "wb"))

## データの読み込み

In [18]:
data = pd.read_pickle('data.pkl')
d_matrix = pickle.load(open("d_matrix.pkl", "rb"))

## 実行してみる

In [19]:
input_lines = []

n = len(data)
input_lines.append(str(n))

for i in range(n):
    row = d_matrix[i]
    input_lines.append(" ".join(map(str, row)))

for i in range(n):
    row = data.iloc[i]
    ab = str(row["arrive_before"])
    aa = str(row["arrive_after"])
    st = str(row["stay_time"])
    pr = str(row["priority"])
    input_lines.append(" ".join([ab, aa, st, pr]))

input_text = "\n".join(input_lines)

In [65]:
res = subprocess.run(["../dist/sa"], input=input_text, capture_output=True, text=True)

In [66]:
path_str = res.stdout.strip().split("\n")[-1]
path = list(map(int, path_str.split(" ")))

In [67]:
current_time = datetime.datetime(2024, 1, 14, 0, 0, 0) + datetime.timedelta(seconds=START_TIME)
m = len(path)
for i in range(m):
    p = path[i]
    print(data.iloc[p]["spot"], current_time.strftime("%H:%M"), end="")
    if i<m-1:
        current_time += datetime.timedelta(seconds=int(data.iloc[p]["stay_time"]))
        if i>=1:
            print(" -> ", end="")
            print(current_time.strftime("%H:%M"))
        else:
            print()
        current_time += datetime.timedelta(seconds=d_matrix[p][path[i + 1]])

京都駅 09:00
三十三間堂 09:20 -> 09:40
音羽山清水寺 10:05 -> 10:35
八坂庚申堂 10:44 -> 10:54
安井金比羅宮 10:58 -> 11:18
八坂神社 11:27 -> 11:42
京都大学中央食堂 12:23 -> 12:53
下鴨神社 13:15 -> 13:35
京都御所 13:51 -> 14:06
錦市場 14:26 -> 14:56
二条城 15:23 -> 15:53
西本願寺 16:34 -> 17:04
京都タワー 17:23 -> 17:53
京都駅 17:54

In [68]:
visualizer_input_lines = []
visualizer_output_lines = []

for i in range(n):
    row = data.iloc[i]
    lat = str(row["latitude"])
    lng = str(row["longitude"])
    pr = str(row["priority"])
    visualizer_input_lines.append(" ".join([lat, lng, pr]))

visualizer_output_lines.append(path_str)

visualizer_input_text = "\n".join(visualizer_input_lines)
visualizer_output_text = "\n".join(visualizer_output_lines)

print(visualizer_input_text)
print()
print(visualizer_output_text)

34.985849 135.7587667 0
34.9946662 135.784661 3
34.9676945 135.7791876 3
35.03937 135.7292431 3
35.0140379 135.7484258 3
35.0036559 135.7785534 1
35.0270213 135.7982058 2
35.0050258 135.764723 3
35.0259202 135.7616454 1
35.0389778 135.7730068 2
35.0592809 135.7525297 2
34.9805982 135.747786 2
34.9878845 135.7717126 2
34.9983244 135.7787304 2
35.0001126 135.7758058 2
35.0114138 135.7944841 3
35.0344943 135.7182634 3
35.03109370000001 135.7138198 1
35.0311737 135.7351227 2
34.987516 135.7472123 3
34.9875205 135.7592518 2
34.9899887 135.773116 2
34.9870986 135.7422732 2
34.99177700000001 135.7516549 2
34.9970888 135.7733663 1
35.0266577 135.7818152 100

0 12 1 13 14 5 25 9 8 7 4 23 20 0


In [69]:
path_history = []
prev = ""
for p in res.stdout.strip().split("\n")[:-2]:
    if p != prev:
        path_history.append(p)
        prev = p
        
pyperclip.copy("\n".join(path_history))